In [1]:
# ========================================
# notebooks/00_Setup.ipynb
# Smart Factory Power Analysis - Setup
# ========================================

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Projects/smart-factory-power')
print(f"✅ 작업 디렉토리: {os.getcwd()}")

Mounted at /content/drive
✅ 작업 디렉토리: /content/drive/MyDrive/Projects/smart-factory-power


In [2]:
# ========================================
# 압축 파일 해제
# ========================================

import zipfile

zip_files = ['train.zip', 'rtu_data_full.csv.zip']
raw_data_path = './data/raw/'

for zip_file in zip_files:
    zip_path = os.path.join(raw_data_path, zip_file)

    if not os.path.exists(zip_path):
        print(f" {zip_file} 파일이 {raw_data_path}에 없습니다.")
        continue

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        internal_files = zip_ref.namelist()
        is_already_extracted = all(os.path.exists(os.path.join(raw_data_path, f)) for f in internal_files)

        if is_already_extracted:
            print(f" {zip_file} 안의 모든 파일이 이미 존재합니다.")
        else:
            print(f" {zip_file}: 압축 해제 중...")
            zip_ref.extractall(raw_data_path)
            print(f" {zip_file}: 압축 풀기 완료")

 train.zip 안의 모든 파일이 이미 존재합니다.
 rtu_data_full.csv.zip 안의 모든 파일이 이미 존재합니다.


In [3]:
# ========================================
# 데이터 확인
# ========================================

import pandas as pd

print("\n=== 데이터 파일 목록 ===")
csv_files = []
for file in sorted(os.listdir('./data/raw/')):
    if file.endswith('.csv'):
        size = os.path.getsize(f'./data/raw/{file}') / (1024**2)
        print(f"{file}: {size:.1f} MB")
        csv_files.append(file)

print("\n" + "="*50)
print("Setup 완료")
print("="*50)


=== 데이터 파일 목록 ===
rtu_data_full.csv: 729.6 MB
submissionfile_sample.csv: 0.0 MB
test.csv: 156.6 MB
train.csv: 4291.0 MB

Setup 완료


In [4]:
# ========================================
# 데이터 로드 (Data Loading)
# ========================================

import pandas as pd
import os

# 데이터 경로
DATA_RAW = './data/raw/'

print("="*60)
print("📊 데이터 로딩 시작")
print("="*60)

def load_csv_with_encoding(filepath, description=""):
    """
    여러 인코딩을 시도하여 CSV 파일 로드

    Parameters:
    -----------
    filepath : str
        파일 경로
    description : str
        파일 설명 (출력용)

    Returns:
    --------
    pd.DataFrame
        로드된 데이터프레임
    """
    encodings = ['utf-8', 'cp949', 'euc-kr', 'latin1']

    if description:
        print(f"\n📁 {description}")

    for encoding in encodings:
        try:
            df = pd.read_csv(filepath, encoding=encoding)
            print(f"   ✅ 로드 성공 (인코딩: {encoding})")
            print(f"   📊 Shape: {df.shape}")
            return df
        except (UnicodeDecodeError, UnicodeError):
            continue

    # 모든 인코딩 실패 시 에러 무시
    print(f"   ⚠️ 자동 감지 실패, errors='ignore' 사용")
    df = pd.read_csv(filepath, encoding='utf-8', errors='ignore')
    print(f"   📊 Shape: {df.shape}")
    return df

# ========================================
# 1. Train 데이터 (메인 학습 데이터)
# ========================================
df_train = load_csv_with_encoding(
    f'{DATA_RAW}train.csv',
    "Train Dataset (학습용 데이터)"
)

# 컬럼명 정리
if 'module(equipment)' in df_train.columns:
    df_train.rename(columns={'module(equipment)': 'module'}, inplace=True)
    print("   🔧 컬럼명 정리: 'module(equipment)' → 'module'")

# ========================================
# 2. Test 데이터 (예측 대상)
# ========================================
df_test = load_csv_with_encoding(
    f'{DATA_RAW}test.csv',
    "Test Dataset (예측 대상)"
)

# 컬럼명 정리
if 'module(equipment)' in df_test.columns:
    df_test.rename(columns={'module(equipment)': 'module'}, inplace=True)
    print("   🔧 컬럼명 정리: 'module(equipment)' → 'module'")

# ========================================
# 3. Submission 샘플 (제출 형식)
# ========================================
df_sample = load_csv_with_encoding(
    f'{DATA_RAW}submissionfile_sample.csv',
    "Submission Sample (제출 형식)"
)

print("\n" + "="*60)
print("✅ 데이터 로딩 완료!")
print("="*60)
print(f"\n📌 로드된 데이터:")
print(f"   - df_train: {df_train.shape}")
print(f"   - df_test: {df_test.shape}")
print(f"   - df_sample: {df_sample.shape}")


📊 데이터 로딩 시작

📁 Train Dataset (학습용 데이터)
   ✅ 로드 성공 (인코딩: utf-8)
   📊 Shape: (27181440, 20)
   🔧 컬럼명 정리: 'module(equipment)' → 'module'

📁 Test Dataset (예측 대상)
   ✅ 로드 성공 (인코딩: cp949)
   📊 Shape: (1048575, 20)
   🔧 컬럼명 정리: 'module(equipment)' → 'module'

📁 Submission Sample (제출 형식)
   ✅ 로드 성공 (인코딩: utf-8)
   📊 Shape: (24, 5)

✅ 데이터 로딩 완료!

📌 로드된 데이터:
   - df_train: (27181440, 20)
   - df_test: (1048575, 20)
   - df_sample: (24, 5)


In [5]:
# ========================================
# 데이터 기본 구조 파악
# ========================================

print("="*60)
print("🔍 데이터 구조 파악")
print("="*60)

# ========================================
# 1. Train 데이터 기본 정보
# ========================================
print("\n" + "="*60)
print("1️⃣ TRAIN 데이터 기본 정보")
print("="*60)

print("\n📋 컬럼 목록:")
print(df_train.columns.tolist())

print("\n📊 데이터 타입:")
print(df_train.dtypes)

print("\n🔢 기본 통계:")
print(df_train.describe())

print("\n👀 첫 10개 행:")
print(df_train.head(10))

🔍 데이터 구조 파악

1️⃣ TRAIN 데이터 기본 정보

📋 컬럼 목록:
['module', 'timestamp', 'localtime', 'operation', 'voltageR', 'voltageS', 'voltageT', 'voltageRS', 'voltageST', 'voltageTR', 'currentR', 'currentS', 'currentT', 'activePower', 'powerFactorR', 'powerFactorS', 'powerFactorT', 'reactivePowerLagging', 'accumActiveEnergy', 'datetime']

📊 데이터 타입:
module                   object
timestamp                 int64
localtime                 int64
operation                 int64
voltageR                float64
voltageS                float64
voltageT                float64
voltageRS               float64
voltageST               float64
voltageTR               float64
currentR                float64
currentS                float64
currentT                float64
activePower             float64
powerFactorR            float64
powerFactorS            float64
powerFactorT            float64
reactivePowerLagging    float64
accumActiveEnergy         int64
datetime                 object
dtype: object

🔢 기본 통계:
 

In [6]:
# ========================================
# 2. Test 데이터 기본 정보
# ========================================
print("\n" + "="*60)
print("2️⃣ TEST 데이터 기본 정보")
print("="*60)

print("\n📋 컬럼 목록:")
print(df_test.columns.tolist())

print("\n👀 첫 10개 행:")
print(df_test.head(10))


2️⃣ TEST 데이터 기본 정보

📋 컬럼 목록:
['module', 'timestamp', 'localtime', 'operation', 'voltageR', 'voltageS', 'voltageT', 'voltageRS', 'voltageST', 'voltageTR', 'currentR', 'currentS', 'currentT', 'activePower', 'powerFactorR', 'powerFactorS', 'powerFactorT', 'reactivePowerLagging', 'accumActiveEnergy', 'datetime']

👀 첫 10개 행:
    module     timestamp     localtime  operation  voltageR  voltageS  \
0  1(PM-3)  1.743490e+12  2.025040e+13          1    218.91    213.07   
1  1(PM-3)  1.743490e+12  2.025040e+13          1    219.60    210.74   
2  1(PM-3)  1.743490e+12  2.025040e+13          1    212.16    217.28   
3  1(PM-3)  1.743490e+12  2.025040e+13          1    217.85    214.18   
4  1(PM-3)  1.743490e+12  2.025040e+13          1    218.86    219.11   
5  1(PM-3)  1.743490e+12  2.025040e+13          1    215.51    214.80   
6  1(PM-3)  1.743490e+12  2.025040e+13          1    211.07    210.81   
7  1(PM-3)  1.743490e+12  2.025040e+13          1    218.75    218.74   
8  1(PM-3)  1.743490

In [7]:
# ========================================
# Test 데이터 상세 확인
# ========================================

print("\n" + "="*60)
print("📊 Test 데이터 상세 분석")
print("="*60)

# 1. 데이터 로드 (UTF-8-SIG)
# utf-8-sig 인코딩을 사용하여 로드하며, 인코딩 에러 발생 시 대체 문자로 처리합니다.
df_test = pd.read_csv('./data/raw/test.csv', encoding='utf-8-sig', encoding_errors='replace')

# 2. 컬럼명 정리
if 'module(equipment)' in df_test.columns:
    df_test.rename(columns={'module(equipment)': 'module'}, inplace=True)

# 3. 모듈명 보정
df_test.loc[df_test['module'].str.contains(r'11\(', na=False), 'module'] = '11(우측분전반1)'
df_test.loc[df_test['module'].str.contains(r'12\(', na=False), 'module'] = '12(4호기)'

# 4. datetime 컬럼 생성 (timestamp는 ms 단위)
df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='ms')

# 5. 결과 확인
print("\n모듈별 데이터 개수:")
print(df_test['module'].value_counts().sort_index())

print("\n시간 범위:")
print(f"시작: {df_test['datetime'].min()}")
print(f"종료: {df_test['datetime'].max()}")

print("\n처음 10개 행의 모듈:")
print(df_test['module'].head(10).tolist())


📊 Test 데이터 상세 분석

모듈별 데이터 개수:
module
1(PM-3)       501121
11(우측분전반1)    501121
12(4호기)        46333
Name: count, dtype: int64

시간 범위:
시작: 2025-04-01 06:46:40
종료: 2025-04-30 08:00:00

처음 10개 행의 모듈:
['1(PM-3)', '1(PM-3)', '1(PM-3)', '1(PM-3)', '1(PM-3)', '1(PM-3)', '1(PM-3)', '1(PM-3)', '1(PM-3)', '1(PM-3)']


In [8]:
# ========================================
# 3. Submission 형식 확인
# ========================================
print("\n" + "="*60)
print("3️⃣ SUBMISSION 형식")
print("="*60)

print("\n📋 컬럼 목록:")
print(df_sample.columns.tolist())

print("\n📄 전체 내용 (24행):")
print(df_sample)



3️⃣ SUBMISSION 형식

📋 컬럼 목록:
['id', 'hourly_pow', 'agg_pow', 'may_bill', 'may_carbon']

📄 전체 내용 (24행):
                        id  hourly_pow         agg_pow        may_bill  \
0      2025-05-01 00:00:00         0.0  동일한 5월의 전체 누적량  동일한 5월 전체 전기요금   
1      2025-05-01 01:00:00         0.0  동일한 5월의 전체 누적량  동일한 5월 전체 전기요금   
2      2025-05-01 02:00:00         0.0  동일한 5월의 전체 누적량  동일한 5월 전체 전기요금   
3      2025-05-01 03:00:00         0.0             NaN               .   
4      2025-05-01 04:00:00         0.0             NaN               .   
5      2025-05-01 05:00:00         0.0             NaN             NaN   
6      2025-05-01 06:00:00         0.0             NaN             NaN   
7      2025-05-01 07:00:00         0.0             NaN             NaN   
8      2025-05-01 08:00:00         0.0             NaN             NaN   
9      2025-05-01 09:00:00         0.0             NaN             NaN   
10     2025-05-01 10:00:00         0.0             NaN             NaN   
11     20

In [9]:
# ========================================
# 4. rtu_data_full.csv 로드 및 분석
# ========================================

print("="*60)
print("🔍 RTU Data Full 로드 및 분석")
print("="*60)

# 파일명 확인
import os
print("\n📁 data/raw/ 파일 목록:")
for f in os.listdir(DATA_RAW):
    print(f"  - {f}")

# RTU 데이터 로드
print("\n📊 rtu_data_full.csv 로드 중...")
df_rtu = pd.read_csv(f'{DATA_RAW}rtu_data_full.csv', encoding='utf-8')

# 컬럼명 정리
if 'module(equipment)' in df_rtu.columns:
    df_rtu.rename(columns={'module(equipment)': 'module'}, inplace=True)

print(f"✅ 로드 완료: {df_rtu.shape}")

# 기본 정보
print("\n" + "="*60)
print("📋 RTU 데이터 기본 정보")
print("="*60)

print(f"\n컬럼: {df_rtu.columns.tolist()}")
print(f"\n데이터 타입:")
print(df_rtu.dtypes)

# 모듈 확인
print("\n" + "="*60)
print("🏭 모듈 정보")
print("="*60)

print(f"\n모듈 수: {df_rtu['module'].nunique()}개")
print(f"\n모듈 목록:")
for module in sorted(df_rtu['module'].unique()):
    print(f"  - {module}")

print(f"\n모듈별 데이터 개수:")
print(df_rtu['module'].value_counts().sort_index())

# 시간 범위
print("\n" + "="*60)
print("⏰ 시간 범위")
print("="*60)

df_rtu['datetime'] = pd.to_datetime(df_rtu['timestamp'], unit='ms')
print(f"\n시작: {df_rtu['datetime'].min()}")
print(f"종료: {df_rtu['datetime'].max()}")
print(f"기간: {(df_rtu['datetime'].max() - df_rtu['datetime'].min()).days}일")

# 샘플 데이터
print("\n처음 10개 행:")
print(df_rtu[['module', 'datetime', 'activePower']].head(10))

# Train과 비교
print("\n" + "="*60)
print("🔄 Train 데이터와 비교")
print("="*60)

rtu_modules = set(df_rtu['module'].unique())
train_modules = set(df_train['module'].unique())

print(f"\nRTU 모듈 수: {len(rtu_modules)}개")
print(f"Train 모듈 수: {len(train_modules)}개")

if rtu_modules == train_modules:
    print("✅ 모듈 완전 일치!")
elif rtu_modules.issubset(train_modules):
    print("✅ RTU는 Train의 부분집합")
    print(f"   RTU에만 있는 모듈: {rtu_modules - train_modules}")
    print(f"   Train에만 있는 모듈: {train_modules - rtu_modules}")
else:
    print("⚠️ 모듈 불일치")

print(f"\nRTU 기간: {df_rtu['datetime'].min()} ~ {df_rtu['datetime'].max()}")
print(f"Train 기간: {df_train['datetime'].min()} ~ {df_train['datetime'].max()}")

print(f"\nRTU 행 수: {len(df_rtu):,}")
print(f"Train 행 수: {len(df_train):,}")
print(f"비율: {len(df_rtu) / len(df_train) * 100:.1f}%")

🔍 RTU Data Full 로드 및 분석

📁 data/raw/ 파일 목록:
  - submissionfile_sample.csv
  - test.csv
  - rtu_data_full.csv.zip
  - train.zip
  - train.csv
  - __MACOSX
  - rtu_data_full.csv

📊 rtu_data_full.csv 로드 중...
✅ 로드 완료: (5425083, 19)

📋 RTU 데이터 기본 정보

컬럼: ['module', 'timestamp', 'localtime', 'operation', 'voltageR', 'voltageS', 'voltageT', 'voltageRS', 'voltageST', 'voltageTR', 'currentR', 'currentS', 'currentT', 'activePower', 'powerFactorR', 'powerFactorS', 'powerFactorT', 'reactivePowerLagging', 'accumActiveEnergy']

데이터 타입:
module                   object
timestamp                 int64
localtime                 int64
operation                 int64
voltageR                float64
voltageS                float64
voltageT                float64
voltageRS               float64
voltageST               float64
voltageTR               float64
currentR                float64
currentS                float64
currentT                float64
activePower             float64
powerFactorR            

In [10]:
# ========================================
# 시간 범위 및 모듈 구조 확인
# ========================================

print("="*60)
print("⏰ 시간 범위 분석")
print("="*60)

# Train 시간 범위
print("\n[TRAIN 데이터]")
print(f"시작: {df_train['datetime'].min()}")
print(f"종료: {df_train['datetime'].max()}")
print(f"기간: {(pd.to_datetime(df_train['datetime'].max()) - pd.to_datetime(df_train['datetime'].min())).days}일")

# Test 시간 범위
print("\n[TEST 데이터]")
print(f"시작: {df_test['datetime'].min()}")
print(f"종료: {df_test['datetime'].max()}")
print(f"기간: {(pd.to_datetime(df_test['datetime'].max()) - pd.to_datetime(df_test['datetime'].min())).days}일")

print("\n" + "="*60)
print("🏭 모듈 구조 분석")
print("="*60)

# 모듈 개수 및 목록
print(f"\n총 모듈 수: {df_train['module'].nunique()}개")
print("\n모듈 목록:")
print(sorted(df_train['module'].unique()))

# 모듈별 데이터 개수
print("\n모듈별 데이터 개수:")
module_counts = df_train['module'].value_counts().sort_index()
print(module_counts)

print("\n모듈별 개수 동일 여부:", module_counts.nunique() == 1)

⏰ 시간 범위 분석

[TRAIN 데이터]
시작: 2024-12-01 00:00:00
종료: 2025-03-31 23:59:55
기간: 120일

[TEST 데이터]
시작: 2025-04-01 06:46:40
종료: 2025-04-30 08:00:00
기간: 29일

🏭 모듈 구조 분석

총 모듈 수: 13개

모듈 목록:
['1(PM-3)', '11(우측분전반1)', '12(4호기)', '13(3호기)', '14(2호기)', '15(예비건조기)', '16(호이스트)', '17(6호기)', '18(우측분전반2)', '2(L-1전등)', '3(분쇄기(2))', '4(분쇄기(1))', '5(좌측분전반)']

모듈별 데이터 개수:
module
1(PM-3)       2090880
11(우측분전반1)    2090880
12(4호기)       2090880
13(3호기)       2090880
14(2호기)       2090880
15(예비건조기)     2090880
16(호이스트)      2090880
17(6호기)       2090880
18(우측분전반2)    2090880
2(L-1전등)      2090880
3(분쇄기(2))     2090880
4(분쇄기(1))     2090880
5(좌측분전반)      2090880
Name: count, dtype: int64

모듈별 개수 동일 여부: True


In [11]:
# ========================================
# 데이터 품질 확인
# ========================================

print("="*60)
print("🔍 데이터 품질 확인")
print("="*60)

# 결측치 확인
print("\n[결측치 확인]")
missing = df_train.isnull().sum()
if missing.sum() == 0:
    print("✅ 결측치 없음")
else:
    print(missing[missing > 0])

# 중복 확인
print("\n[중복 확인]")
duplicates = df_train.duplicated().sum()
print(f"중복 행: {duplicates:,}개")

# operation 확인
print("\n[가동 상태 확인]")
print(df_train['operation'].value_counts())
print(f"항상 가동 중: {df_train['operation'].std() == 0}")

# 시간 간격 확인 (5초 단위 검증)
print("\n[시간 간격 확인]")
df_train_sorted = df_train.sort_values(['module', 'timestamp'])
time_diff = df_train_sorted.groupby('module')['timestamp'].diff() / 1000  # 초 단위
print(f"시간 간격 (초):")
print(time_diff.value_counts().head(10))

🔍 데이터 품질 확인

[결측치 확인]
✅ 결측치 없음

[중복 확인]
중복 행: 0개

[가동 상태 확인]
operation
1    27181440
Name: count, dtype: int64
항상 가동 중: True

[시간 간격 확인]
시간 간격 (초):
timestamp
5.0    27172067
0.0        9360
Name: count, dtype: int64


In [13]:
# ========================================
# 모듈 1 데이터 추출 및 저장
# ========================================

import pandas as pd
import os

print("="*60)
print("💾 모듈 1 데이터 저장")
print("="*60)

# 저장 경로 생성
module1_dir = './data/processed/module1/'
os.makedirs(module1_dir, exist_ok=True)

# 모듈 1 필터링
print("\n[모듈 1 필터링 중...]")
df_train_m1 = df_train[df_train['module'] == '1(PM-3)'].copy()
df_test_m1 = df_test[df_test['module'] == '1(PM-3)'].copy()

print(f"Train 모듈 1: {df_train_m1.shape}")
print(f"Test 모듈 1: {df_test_m1.shape}")

# Train 저장 (utf-8)
print("\n[Train 저장 중...]")
train_path = os.path.join(module1_dir, 'train.csv')
df_train_m1.to_csv(train_path, index=False, encoding='utf-8')
print(f"✅ 저장 완료: {train_path}")
print(f"   행 수: {len(df_train_m1):,}")
print(f"   컬럼 수: {len(df_train_m1.columns)}")

# 파일 크기 확인
file_size = os.path.getsize(train_path) / (1024**2)
print(f"   파일 크기: {file_size:.1f} MB")

# Test 저장 (utf-8-sig)
print("\n[Test 저장 중...]")
test_path = os.path.join(module1_dir, 'test.csv')
df_test_m1.to_csv(test_path, index=False, encoding='utf-8-sig')
print(f"✅ 저장 완료: {test_path}")
print(f"   행 수: {len(df_test_m1):,}")
print(f"   컬럼 수: {len(df_test_m1.columns)}")

# 파일 크기 확인
file_size = os.path.getsize(test_path) / (1024**2)
print(f"   파일 크기: {file_size:.1f} MB")

# 저장된 파일 목록 확인
print("\n" + "="*60)
print("📁 저장된 파일 목록")
print("="*60)
for file in os.listdir(module1_dir):
    filepath = os.path.join(module1_dir, file)
    size = os.path.getsize(filepath) / (1024**2)
    print(f"  - {file}: {size:.1f} MB")

# 로드 테스트
print("\n" + "="*60)
print("🔍 저장 확인 (로드 테스트)")
print("="*60)

test_load_train = pd.read_csv(train_path, nrows=5)
test_load_test = pd.read_csv(test_path, nrows=5, encoding='utf-8-sig')

print("\nTrain 처음 5개 행:")
print(test_load_train[['module', 'datetime', 'activePower']].head())

print("\nTest 처음 5개 행:")
print(test_load_test[['module', 'datetime', 'activePower']].head())

print("\n✅ 모듈 1 데이터 저장 완료!")
print(f"   저장 위치: {module1_dir}")

💾 모듈 1 데이터 저장

[모듈 1 필터링 중...]
Train 모듈 1: (2090880, 20)
Test 모듈 1: (501121, 20)

[Train 저장 중...]
✅ 저장 완료: ./data/processed/module1/train.csv
   행 수: 2,090,880
   컬럼 수: 20
   파일 크기: 315.3 MB

[Test 저장 중...]
✅ 저장 완료: ./data/processed/module1/test.csv
   행 수: 501,121
   컬럼 수: 20
   파일 크기: 77.9 MB

📁 저장된 파일 목록
  - train.csv: 315.3 MB
  - test.csv: 77.9 MB

🔍 저장 확인 (로드 테스트)

Train 처음 5개 행:
    module             datetime  activePower
0  1(PM-3)  2024-12-01 00:00:00      2961.61
1  1(PM-3)  2024-12-01 00:00:05      3017.48
2  1(PM-3)  2024-12-01 00:00:10      2408.01
3  1(PM-3)  2024-12-01 00:00:15      3289.33
4  1(PM-3)  2024-12-01 00:00:20      3069.31

Test 처음 5개 행:
    module             datetime  activePower
0  1(PM-3)  2025-04-01 06:46:40      2561.53
1  1(PM-3)  2025-04-01 06:46:40      3313.54
2  1(PM-3)  2025-04-01 06:46:40      3428.81
3  1(PM-3)  2025-04-01 06:46:40      2483.51
4  1(PM-3)  2025-04-01 06:46:40      3458.31

✅ 모듈 1 데이터 저장 완료!
   저장 위치: ./data/processed/module1/
